This script re-scales the Hansen data mask to the MODIS LAI resolution.

In [36]:
# setting up all the packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import torch
import torch.nn as nn

import rasterio
from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform, reproject
import geopandas as gpd
from rasterio.mask import mask
import rioxarray as rxr

#from utils import * 

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

plt.rcParams['savefig.dpi'] = 400
plt.rcParams['font.size'] = 13
plt.rcParams["legend.frameon"] = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
# call in the MODIS LAI product from the LEAP data catalogue

store = 'https://nyu1.osn.mghpcc.org/leap-pangeo-pipeline/MODIS_LAI/MODIS_LAI.zarr'
ds = xr.open_dataset(store, engine='zarr', chunks={})

# get resolution of MODIS LAI data product
# get difference between lat and lon
res_lat = ds.lat.diff("lat").mean().item()
res_lon = ds.lon.diff("lon").mean().item()


print("Resolution (lat):", res_lat)
print("Resolution (lon):", res_lon)

Resolution (lat): -0.05000000074505806
Resolution (lon): 0.05000000074505806


In [38]:
# open the Hansen data mask dataset
hansen_datamask_path = "/home/jovyan/MLEAEEE4000-DroughtAmazon2/data/mdd_extent_TCscale.tif"   # Hansen data, raster to be resampled
hansen = rioxarray.open_rasterio(hansen_datamask_path)

In [39]:
# set raster path 
hansen_datamask_path = "/home/jovyan/MLEAEEE4000-DroughtAmazon2/data/mdd_extent_TCscale.tif"   # Hansen data, raster to be resampled

# Load raster
hansen = rxr.open_rasterio(hansen_datamask_path)

# set desired new resolution to the resolution of the MODIS LAI product, defined above
new_res = (res_lon, res_lat)

# resampling using "nearest" method, best for categorical data
hansen_resampled = hansen.rio.reproject(
    hansen.rio.crs,
    resolution=new_res,
    resampling="nearest"
)


In [40]:
# export the resampled datamask
hansen_resampled.rio.to_raster("/home/jovyan/MLEAEEE4000-DroughtAmazon2/data/hansen_datamask.tif")